In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm

In [2]:

# Mediapipe setup
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

pose = mp_pose.Pose(static_image_mode=False)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)


In [3]:
# Constants
FPS = 30
FRAMES_TO_SKIP = 30  # Skip first and last second
FRAMES_TO_KEEP = 60  # Middle 60 frames (2 seconds)
ROOT_DIR = "BDSL_Video_draft"        # Input video dataset
SAVE_DIR = "BDSL_Video_Numpy"  # Output numpy arrays


In [4]:

def extract_landmark_features(results_pose, results_hands):
    features = []

    if results_pose.pose_landmarks:
        lm = results_pose.pose_landmarks.landmark
        try:
            nose = np.array([lm[0].x, lm[0].y, lm[0].z])
            left_wrist = np.array([lm[15].x, lm[15].y, lm[15].z])
            right_wrist = np.array([lm[16].x, lm[16].y, lm[16].z])
            dist_left = np.linalg.norm(left_wrist - nose)
            dist_right = np.linalg.norm(right_wrist - nose)
        except IndexError:
            dist_left = dist_right = 0.0
    else:
        dist_left = dist_right = 0.0

    features.extend([dist_left, dist_right])

    hands_data = []
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            one_hand = []
            for lm in hand_landmarks.landmark:
                one_hand.extend([lm.x, lm.y, lm.z])
            hands_data.append(one_hand)

    if len(hands_data) == 1:
        hands_data.append([0.0]*63)
    elif len(hands_data) == 0:
        hands_data = [[0.0]*63, [0.0]*63]

    features.extend(hands_data[0])
    features.extend(hands_data[1])

    return np.array(features, dtype=np.float32)

def process_video(video_path, save_path_base):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"[INFO] Processing: {os.path.basename(video_path)} | Total frames: {total_frames}")

    if total_frames < FRAMES_TO_SKIP + FRAMES_TO_KEEP:
        print(f"[SKIP] Not enough frames in: {video_path}")
        cap.release()
        return

    cap.set(cv2.CAP_PROP_POS_FRAMES, FRAMES_TO_SKIP)
    os.makedirs(save_path_base, exist_ok=True)

    for i in range(FRAMES_TO_KEEP):
        ret, frame = cap.read()
        if not ret:
            print(f"[WARN] Could not read frame {i}")
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results_pose = pose.process(frame_rgb)
        results_hands = hands.process(frame_rgb)

        features = extract_landmark_features(results_pose, results_hands)
        save_path = os.path.join(save_path_base, f"frame_{i:03d}.npy")
        np.save(save_path, features)

    cap.release()
    print(f"[✅] Saved frames to {save_path_base}")

def main():
    os.makedirs(SAVE_DIR, exist_ok=True)

    for class_name in os.listdir(ROOT_DIR):
        class_path = os.path.join(ROOT_DIR, class_name)
        if not os.path.isdir(class_path):
            continue

        # Look for video files inside this class folder
        video_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.avi', '.mp4'))]
        
        # Skip if no videos found
        if not video_files:
            print(f"[SKIP] No videos found in class folder: {class_name}")
            continue

        print(f"[INFO] {len(video_files)} video(s) found in '{class_name}'.")

        for video_name in video_files:
            video_path = os.path.join(class_path, video_name)
            save_path = os.path.join(SAVE_DIR, class_name, os.path.splitext(video_name)[0])
            process_video(video_path, save_path)

    print("[DONE] All videos processed.")



if __name__ == "__main__":
    print("[START] Extracting landmarks to numpy arrays...")
    main()


[START] Extracting landmarks to numpy arrays...
[INFO] 212 video(s) found in 'kaj'.
[INFO] Processing: Student_10_1668499363.avi | Total frames: 120
[✅] Saved frames to BDSL_Video_Numpy\kaj\Student_10_1668499363
[INFO] Processing: Student_10_1668499379.avi | Total frames: 120
[✅] Saved frames to BDSL_Video_Numpy\kaj\Student_10_1668499379
[INFO] Processing: Student_10_1668499390.avi | Total frames: 120
[✅] Saved frames to BDSL_Video_Numpy\kaj\Student_10_1668499390
[INFO] Processing: Student_10_1668499398.avi | Total frames: 120
[✅] Saved frames to BDSL_Video_Numpy\kaj\Student_10_1668499398
[INFO] Processing: Student_11_1668574297.avi | Total frames: 120
[✅] Saved frames to BDSL_Video_Numpy\kaj\Student_11_1668574297
[INFO] Processing: Student_11_1668574333.avi | Total frames: 120
[✅] Saved frames to BDSL_Video_Numpy\kaj\Student_11_1668574333
[INFO] Processing: Student_11_1668574342.avi | Total frames: 120
[✅] Saved frames to BDSL_Video_Numpy\kaj\Student_11_1668574342
[INFO] Processing: S